**[결측치 처리]**
- 시계열데이터이므로 결측치를 평균값 보다는 보간법으로 채워주는 것이 좋음 <br> (그래야 t-1시점과 t+1시점의 값 사이에 smooth 하게 이어줄 수 있음) <br>
(특히, 우리나라 강수량은 여름 며칠에만 집중적이어서 평균값이 전체 특성을 반영하지 못할 수 있음)
- 공용량(ecpc) 이상치는 저수량(sfw) 결측치, 이상치를 먼저 채운 뒤에 '244.0 - sfw' 값으로 대체

**[이상치 탐색]**
- 총방류량(tototf), 유입량(inf) 이상치
  - 2013-09-16, 2018-06-08, 2018-07-05 : 두 변수 보간법 등으로 변경해주기
  - 2020-08-22 18:50:00 : 방류량0으로 변경해주기
  - 2020-08-22 19:00:00 : 유입량 bfill로 변경해주기

## 데이터 불러오기

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns 
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
filepath='/content/drive/Shareddrives/팔당댐/combined_data/combined_data(2012-2022).csv'

In [4]:
#인덱스type을 datetime형으로 변환하여 데이터프레임 생성
df=pd.read_csv(filepath, index_col='ymdhm', parse_dates=['ymdhm'])
df.drop('Unnamed: 0',axis=1,inplace=True)
df.index

DatetimeIndex(['2012-05-01 00:00:00', '2012-05-01 00:10:00',
               '2012-05-01 00:20:00', '2012-05-01 00:30:00',
               '2012-05-01 00:40:00', '2012-05-01 00:50:00',
               '2012-05-01 01:00:00', '2012-05-01 01:10:00',
               '2012-05-01 01:20:00', '2012-05-01 01:30:00',
               ...
               '2022-07-18 22:20:00', '2022-07-18 22:30:00',
               '2022-07-18 22:40:00', '2022-07-18 22:50:00',
               '2022-07-18 23:00:00', '2022-07-18 23:10:00',
               '2022-07-18 23:20:00', '2022-07-18 23:30:00',
               '2022-07-18 23:40:00', '2022-07-18 23:50:00'],
              dtype='datetime64[ns]', name='ymdhm', length=276336, freq=None)

In [5]:
df

,swl,inf,sfw,ecpc,tototf,tide_level,wl_1018662,fw_1018662,wl_1018680,fw_1018680,wl_1018683,fw_1018683,wl_1019630,fw_1019630,rf_10184100,rf_10184110,rf_10184140
ymdhm,,,,,,,,,,,,,,,,,
2012-05-01 00:00:00,24.800,555.00,219.07,24.93,555.00,445.0,310.7,469.05,300.2,0.0,290.0,729.80,275.3,540.18,0.0,0.0,0.0
2012-05-01 00:10:00,24.794,464.60,218.86,25.15,562.90,449.0,314.7,498.00,300.2,0.0,290.0,731.48,275.3,540.18,0.0,0.0,0.0
2012-05-01 00:20:00,24.789,478.10,218.69,25.31,576.40,451.0,313.7,490.68,301.2,0.0,290.0,726.42,275.3,540.18,0.0,0.0,0.0
2012-05-01 00:30:00,24.789,464.80,218.69,25.31,563.10,452.0,311.7,476.21,301.2,0.0,290.0,726.42,276.3,552.17,0.0,0.0,0.0
2012-05-01 00:40:00,24.789,478.10,218.69,25.31,576.40,450.0,311.7,476.21,301.2,0.0,291.0,707.17,277.3,564.29,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-18 23:10:00,25.040,259.23,212.86,31.14,259.23,510.0,0.0,319.84,0.0,NaN,0.0,-456.41,0.0,974.40,0.0,0.0,0.0
2022-07-18 23:20:00,25.040,260.46,212.86,31.14,260.46,492.0,0.0,314.01,0.0,NaN,0.0,-717.30,0.0,1006.88,0.0,0.0,0.0
2022-07-18 23:30:00,25.040,259.37,212.86,31.14,259.37,475.0,0.0,387.55,0.0,NaN,0.0,-843.37,0.0,1039.90,0.0,0.0,0.0


## 결측치 처리
- 보간법으로 결측치 채우기
- ```Pandas.Series.Interpolate``` [설명 링크](https://pandas.pydata.org/docs/reference/api/pandas.Series.interpolate.html)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 276336 entries, 2012-05-01 00:00:00 to 2022-07-18 23:50:00
Data columns (total 17 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   swl          275593 non-null  float64
 1   inf          275593 non-null  float64
 2   sfw          275593 non-null  float64
 3   ecpc         275593 non-null  float64
 4   tototf       275593 non-null  float64
 5   tide_level   271409 non-null  float64
 6   wl_1018662   276277 non-null  float64
 7   fw_1018662   259956 non-null  float64
 8   wl_1018680   276277 non-null  float64
 9   fw_1018680   79487 non-null   float64
 10  wl_1018683   276277 non-null  float64
 11  fw_1018683   275057 non-null  float64
 12  wl_1019630   276277 non-null  float64
 13  fw_1019630   276277 non-null  float64
 14  rf_10184100  276336 non-null  float64
 15  rf_10184110  276336 non-null  float64
 16  rf_10184140  276336 non-null  float64
dtypes: float64(17)
memory usage: 37.9

In [7]:
df.isnull().sum()

swl               743
inf               743
sfw               743
ecpc              743
tototf            743
tide_level       4927
wl_1018662         59
fw_1018662      16380
wl_1018680         59
fw_1018680     196849
wl_1018683         59
fw_1018683       1279
wl_1019630         59
fw_1019630         59
rf_10184100         0
rf_10184110         0
rf_10184140         0
dtype: int64

In [8]:
df.interpolate(method='linear', inplace=True)

In [9]:
df.isnull().sum()

swl            0
inf            0
sfw            0
ecpc           0
tototf         0
tide_level     0
wl_1018662     0
fw_1018662     0
wl_1018680     0
fw_1018680     0
wl_1018683     0
fw_1018683     0
wl_1019630     0
fw_1019630     0
rf_10184100    0
rf_10184110    0
rf_10184140    0
dtype: int64

## 이상치 처리

### swl(현재수위), sfw(현재저수량), ecpc(공용량)
저수량(sfw) + 공용량(ecpc) = 총저수용량 = 244.0 이므로 
- 현재수위(swl)과 저수량(sfw)만 보간법으로 이상치 변환해주고 
- 공용량(ecpc)는 '244.0-sfw' 값으로 변환하기

In [10]:
# swl, sfw 이상치 -> 결측치로 바꿔주기
dt_range=df['swl'][df['swl']==0].index

for i in range(len(dt_range)):
  df.loc[dt_range[i]]['swl']=np.nan

for i in range(len(dt_range)):
  df.loc[dt_range[i]]['sfw']=np.nan

In [11]:
# swl, sfw 결측치 -> 보간법으로 채우기
df['swl'].interpolate(method='linear', inplace=True)
df['sfw'].interpolate(method='linear', inplace=True)

In [12]:
df[['swl','sfw']].isnull().sum()

swl    0
sfw    0
dtype: int64

ecpc 이상치: ecpc = 244-sfw로 바꾸기

In [13]:
for i in range(len(dt_range)):
  df.loc[dt_range[i]]['ecpc']=244.0-df.loc[dt_range[i]]['sfw']

In [15]:
df[['ecpc']].isnull().sum()

ecpc    0
dtype: int64

In [14]:
df['ecpc'][df['ecpc']>240] # 이상치 없음

Series([], Name: ecpc, dtype: float64)

### 총방류량(tototf), 유입량(inf) 
- 2013-09-16, 2018-06-08, 2018-07-05 : 두 변수 보간법 등으로 변경해주기
- 2020-08-22 18:50:00 : 방류량0으로 변경해주기
- 2020-08-22 19:00:00 : 유입량 bfill로 변경해주기

In [16]:
# 이상치 -> 결측치로 바꿔주기
dt_range=['2013-09-16 04:40:00', '2018-06-08 16:00:00', '2018-07-05 16:40:00']
for i in range(len(dt_range)):
  df.loc[dt_range[i]]['tototf']=np.nan
  df.loc[dt_range[i]]['inf']=np.nan

In [17]:
df['tototf'].interpolate(method='linear', inplace=True)
df['inf'].interpolate(method='linear', inplace=True)

In [18]:
df.loc['2020-08-22 18:50:00']['tototf']=0.0

In [19]:
df.loc['2020-08-22 19:00:00']['inf']=np.nan
df.fillna(method='bfill', inplace=True)
df.isnull().sum()

swl            0
inf            0
sfw            0
ecpc           0
tototf         0
tide_level     0
wl_1018662     0
fw_1018662     0
wl_1018680     0
fw_1018680     0
wl_1018683     0
fw_1018683     0
wl_1019630     0
fw_1019630     0
rf_10184100    0
rf_10184110    0
rf_10184140    0
dtype: int64

## 데이터 csv로 저장

In [20]:
# df.to_csv('/content/drive/Shareddrives/팔당댐/combined_data/filled_data(2012-2022).csv')